In [11]:

# general
import wandb
from MulticoreTSNE import MulticoreTSNE as TSNE
from tsnecuda import TSNE
import re
from sklearn import metrics
from tqdm import tqdm
import Utils
import transformer.Constants as Constants
import torch.optim as optim
import torch.nn as nn
import torch
import time
import argparse
import numpy as np
import pandas as pd
import math
# import matplotlib.pyplot as plt
import os
import shutil
import pickle

import Main
from transformer.Models import TEEDAM, align
# plotly
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt

# import custom libraries
import sys
# sys.path.append("C:\\DATA\\Tasks\\lib\\hk")
# import hk_utils

# folder paths
ADD_DATA = "C:\\DATA\\data\\raw\\mimic4\\lookup\\"
ADD_DATA_proc = "C:/DATA/data/processed/"


PATH_PAPER = "C:\\DATA\\Tasks\\220704\\Alternate-Transactions-Articles-LaTeX-template\\images\\"


PATH_SYS = "/mlodata1/hokarami/tedam/"


In [17]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# plotly
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np

In [2]:

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# torch.cuda.empty_cache()
# torch.cuda.memory_allocated()
# torch.cuda.memory_reserved()

# from hk_pytorch import save_checkpoint,load_checkpoint
# import hk_pytorch


# from custom2 import myparser


# wandb.login()
api = wandb.Api()

os.environ["WANDB_API_KEY"] = "0f780ac8a470afe6cb7fc474ff3794772c660465"

In [7]:

def build_df(out, opt, X_tsne=None, TSNE_LIMIT=5000):
    if 'y_state_true' in out:

        y_state_pred = out['y_state_pred']
        y_state_true = out['y_state_true']
        y_state_score = out['y_state_score']
    else:
        y_state_pred = None
        y_state_true = None
        y_state_score = None

    y_pred = out['y_pred']
    y_true = out['y_true']
    y_score = out['y_score']

    df = pd.DataFrame()

    if X_tsne is not None:
        df['x'] = X_tsne[:, 0]
        df['y'] = X_tsne[:, 1]
    else:
        df['x'] = y_state_true
        TSNE_LIMIT = 100000000

    # df['x']=X_te_tsne[:,0]
    # df['y']=X_te_tsne[:,1]

    # df['x']=X_dam_tsne[:,0]
    # df['y']=X_dam_tsne[:,1]

    df['color'] = 0
    df['id'] = np.arange(len(df))

    if y_state_true is not None:

        TP = (y_state_true[:TSNE_LIMIT]*y_state_pred[:TSNE_LIMIT]) == 1
        FN = (y_state_true[:TSNE_LIMIT]-y_state_pred[:TSNE_LIMIT]) == 1
        FP = (y_state_true[:TSNE_LIMIT]-y_state_pred[:TSNE_LIMIT]) == -1

        TN = (y_state_true[:TSNE_LIMIT]+y_state_pred[:TSNE_LIMIT]) == 0

        FP_FN = (y_state_true[:TSNE_LIMIT]+y_state_pred[:TSNE_LIMIT]) == 1
        TP_TN = (y_state_true[:TSNE_LIMIT]-y_state_pred[:TSNE_LIMIT]) == 0

        # df.loc[TN, 'color']='True Negatives'
        df.loc[TP, 'color'] = 'True Positives'
        df.loc[FN, 'color'] = 'False Negatives'
        df.loc[FP, 'color'] = 'False Positives'
        df.loc[TN, 'color'] = 'True Negatives'

        df.loc[TP_TN, 'color_true_pred'] = 'True Predicted'
        df.loc[FP_FN, 'color_true_pred'] = 'False Predicted'

        df.loc[y_state_true[:TSNE_LIMIT].astype(
            bool).flatten(), 'color_true'] = 'Positive Samples'
        df.loc[~y_state_true[:TSNE_LIMIT].astype(
            bool).flatten(), 'color_true'] = 'Negative Samples'

        df.loc[y_state_pred[:TSNE_LIMIT].astype(
            bool).flatten(), 'color_pred'] = 'Positive Predicted'
        df.loc[~y_state_pred[:TSNE_LIMIT].astype(
            bool).flatten(), 'color_pred'] = 'Negative Predicted'

        # df.loc[y_state_pred[:TSNE_LIMIT].astype(bool).flatten(), 'color_true_pred']='Positive Predicted'
        # df.loc[~y_state_pred[:TSNE_LIMIT].astype(bool).flatten(), 'color_true_pred']='Negative Predicted'

    df['i_b'] = df['id'].apply(lambda x: int(x / opt.batch_size))
    df['i'] = df['id'].apply(lambda x: x % opt.batch_size)

    t_max = np.concatenate([t.max(1)[0] for t in out['event_time_list']])

    df['color_t_max'] = t_max
    if len(out['list_log_sum']) > 0:
        df['color_log_sum'] = np.concatenate(
            out['list_log_sum'], axis=0) / t_max
        df['color_integral_'] = np.concatenate(
            out['list_integral_'], axis=0) / t_max
    return df


def event2mat(event_type, event_time, P):

    m = event_type.sum(1) > 0  # False are masked
    e = event_type[m, :]
    t = event_time[m]
    # print(t)

    indices = e.nonzero()
    indices[:, 0] = t[indices[:, 0]].int()
    # print(t[indices[:,0]])
    # print(indices[:,0])

    M = torch.zeros((P, e.shape[-1]))
    M[indices[:, 0], indices[:, 1]] = 1

    return M.detach().cpu().numpy().transpose()  # [M,P]


# Project is specified by <entity/project-name>
def dl_runs(all_runs, selected_tag=None):

    summary_list, config_list, name_list, path_list = [], [], [], []
    for run in all_runs:

        if (selected_tag not in run.tags) and (selected_tag is not None):
            continue
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.

        config_list.append(
            {k: v for k, v in run.config.items()
             if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)
        path_list.append(run.path)

    runs_df = pd.DataFrame({
        "summary": summary_list,
        "config": config_list,
        "name": name_list,
        "path": path_list

    })

    # runs_df.to_csv("project.csv")
    return runs_df


def read_from_wandb(run_path, consider_sample_labels=False):

    run = api.run(run_path)

    # for file in run.files():
    #     file.download(replace=True,root=f'./local/{run_path}/')
    for file in run.files():
        if 'best_model' in file.name or 'opt.pkl' in file.name:

            try:
                file.download(replace=True, root=f'.local/{run_path}')

            except:
                print('ERROR')
                pass
    opt = pickle.load(open(f'.local/{run_path}/opt.pkl', 'rb'))

    if consider_sample_labels:
        opt.sample_label = True

    opt = Main.config(opt, justLoad=True)
    if not hasattr(opt, 'diag_offset'):
        opt.diag_offset = 1
        print('ATTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT')
    checkpoint = torch.load(f'.local/{run_path}/best_model.pkl')

    model = TEEDAM(
        n_marks=opt.num_marks,
        TE_config=opt.TE_config,
        DAM_config=opt.DAM_config,
        NOISE_config=opt.NOISE_config,

        CIF_config=opt.CIF_config,
        next_time_config=opt.next_time_config,
        next_type_config=opt.next_type_config,
        label_config=opt.label_config,

        demo_config=opt.demo_config,

        device=opt.device,
        diag_offset=opt.diag_offset
    )

    _ = model.to(opt.device)
    _ = model.load_state_dict(checkpoint['model_state_dict'], strict=False)
    _ = model.eval()

    return model, opt, run


def find_knn_pids(X, id_origin, n_knn=10):

    # X [N,d] np.array
    r = np.sqrt(((X-X[id_origin])**2).sum(-1))
    # knn_pids = list(np.argpartition(r,n_knn)[:n_knn])
    knn_pids = np.argsort(r)[:n_knn]
    if id_origin not in knn_pids:
        print('bad ', id_origin, knn_pids)

    else:
        # knn_pids.remove(id_origin)
        aa = 1

    # print(X.shape)
    # term

    # x0 = df.iloc[id_origin]['x']
    # y0 = df.iloc[id_origin]['y']

    # df['r'] = df.apply(lambda row: ( (row['x']-x0)**2 + (row['y']-y0)**2  ), axis=1)

    # knn_pids = list(df.nsmallest(n_knn,'r').index)

    return knn_pids


def cal_similarity(df, out, pid_origin, knn_pids):

    list_summary = []
    list_t_max = []
    for pid in knn_pids:

        i_b = df.iloc[pid]['i_b']
        i = df.iloc[pid]['i']

        ev = out['event_type_list'][i_b][i]
        t = out['event_time_list'][i_b][i]
        st = out['state_time_list'][i_b][i]

        P = st.int().max().item() + 1

        M = event2mat(ev, t, P)

        # if ev.sum()==0:
        #     print(pid)
        #     continue
        #     # print(t.shape,ev.sum())
        vector = M.sum(1)/M.shape[1]*24

        # if (vector.sum()==0):
        #     print(M.shape,t.max(),ev.sum(),vector.sum(),'binjour',st.max())
        #     # print(pid_origin,pid)
        #     # print('shit')
        #     if pid==pid_origin:
        #         print('WTF ',pid)
        #         raise 'pid_origin has not pattern!'
        #     # bad_ids.append(pid)
        #     continue

        list_summary.append(vector)
        list_t_max.append(t.max().item())

    dotp = [np.dot(j, list_summary[0])/(np.linalg.norm(j) *
                                        np.linalg.norm(list_summary[0])) for j in list_summary]
    sim_score = np.mean([x for x in dotp if ~np.isnan(x)]
                        )  # remove nan elements

    if np.isnan(sim_score.sum()):
        print('he', sim_score.sum())

    # print('ff',list_t_max)
    list_t_max = [x for x in list_t_max if x > 0]
    temp = np.array(list_t_max)
    t_max_mae = np.mean(np.abs(temp-temp[0]))
    # t_max_score = [x-list_t_max[0] for x in list_t_max[1:]]

    return sim_score, list_summary, t_max_mae


def remove_ids_with_no_pattern(df, out):

    bad_pids = []
    for pid in df.id:

        i_b = df.iloc[pid]['i_b']
        i = df.iloc[pid]['i']

        # ev = out['event_type_list'][i_b][i]
        t = out['event_time_list'][i_b][i]
        # st=out['state_time_list'][i_b][i]

        if t.max() == 0:
            bad_pids.append(pid)

    print(f"{len(bad_pids)} patients were removed due to no existing pattern")
    df_out = df.loc[~df.id.isin(bad_pids)]
    df_out.loc[:, 'id'] = np.arange(len(df_out))

    return df_out, bad_pids



In [9]:

def valid_epoch_tsne(model, validation_data, pred_loss_func, opt):
    """ Epoch operation in evaluation phase. """
    example = {}
    model.eval()

    out = dict()

    total_event_ll = 1  # cumulative event log-likelihood
    total_event_rate = 1  # cumulative number of correct prediction
    total_num_event = 1  # number of total events
    total_num_pred = 1  # number of predictions, total=tqdm_len

    total_time_sse = 1  # cumulative time prediction squared-error
    total_time_sae = 1  # cumulative time prediction squared-error
    total_time_sse_norm = 1  # cumulative time prediction squared-error

    total_label_state = 0  # cumulative time prediction squared-error

    event_type_list = []
    event_time_list = []
    next_event_type_list = []

    non_pad_mask_list = []

    time_gap_true = []
    time_gap_pred = []
    X_enc = []
    y_pred_list = []
    y_true_list = []
    y_score_list = []

    y_event_pred_list = []
    y_event_true_list = []
    y_event_score_list = []

    y_state_pred_list = []
    y_state_true_list = []
    y_state_score_list = []
    r_enc_list = []
    r_enc_list2 = []
    masks_list = []

    state_mod_list = []
    state_time_list = []
    state_value_list = []

    y_pred_stupid_list = []
    n_classes = model.n_marks

    list_intens_at_samples = []
    list_taus = []
    list_true_intens_at_evs = []
    list_log_sum = []
    list_integral_ = []

    list_TE_att = []
    list_DAM_att = []

    dict_metrics = {}

    with torch.no_grad():
        for batch in tqdm(validation_data, mininterval=2,
                          desc='  - (Testing)   ', leave=False):
            """ prepare data """

            batch = [x.to(opt.device) for x in batch]
            # batch = map(lambda x: x.to(opt.device), batch)

            state_data = []
            state_label = None
            # if opt.event_enc:
            event_time, time_gap, event_type = batch[:3]
            if opt.state:
                state_time, state_value, state_mod = batch[3:6]
                state_mod_list.append(state_mod.detach().cpu())
                state_time_list.append(state_time.detach().cpu())
                state_value_list.append(state_value.detach().cpu())

                state_data = batch[3:6]
            if opt.sample_label:
                state_time, state_value, state_mod = batch[3:6]
                state_label = batch[6]
            if opt.demo:
                state_data.append(batch[-1])

            non_pad_mask = Utils.get_non_pad_mask(event_type).squeeze(2)

            enc_out = model(event_type, event_time, state_data=state_data)

            a = (non_pad_mask.sum(1)-1).int().tolist()
            r = enc_out[[i for i in range(enc_out.shape[0])], a, :]
            r_enc_list.append(r.detach().cpu())

            # r_enc_list.append(enc_out.detach().cpu()[:,-1,:])
            # r_enc_list2.append(enc_out.detach().cpu()[:,-2,:])
            # next_list2.append(event_type.detach().cpu()[:])
            event_type_list.append(event_type.detach().cpu())
            if (event_type.sum(-1).sum(-1) == 0).any() == 0:
                print("TOO BAD")
            event_time_list.append(event_time.detach().cpu())

            non_pad_mask_list.append(non_pad_mask)

            total_num_pred += non_pad_mask.sum().item()
            masks_list.append(
                non_pad_mask[:, 1:].flatten().bool().detach().cpu())  # [*, C]

            # CIF decoder
            if hasattr(model, 'event_decoder'):
                log_sum, integral_ = model.event_decoder(
                    enc_out, event_time, event_type, non_pad_mask)

                list_intens_at_samples.append(
                    model.event_decoder.intens_at_samples.detach().cpu())
                list_taus.append(model.event_decoder.taus.detach().cpu())
                list_true_intens_at_evs.append(
                    model.event_decoder.true_intens_at_evs.detach().cpu())
                list_log_sum.append(log_sum.detach().cpu())
                list_integral_.append(integral_.detach().cpu())

                total_event_ll += torch.sum(log_sum - integral_)

                y_event_score_list.append(torch.flatten(
                    model.event_decoder.intens_at_evs, end_dim=1).detach().cpu())  # [*, n_cif]

            if hasattr(model, 'TE'):
                list_TE_att.append(model.TE.self_attn)
            if hasattr(model, 'DAM'):
                list_DAM_att.append(model.DAM.att)

            # next type prediction
            if hasattr(model, 'pred_next_type'):
                pred_loss, pred_num_event, (y_pred, y_true, y_score, masks) = opt.type_loss(
                    model.y_next_type, event_type, pred_loss_func)
                # total_loss.append(pred_loss)
                y_pred_list.append(torch.flatten(
                    y_pred, end_dim=1).detach().cpu())  # [*]
                y_true_list.append(torch.flatten(
                    y_true, end_dim=1).detach().cpu())  # [*]
                y_score_list.append(torch.flatten(
                    y_score, end_dim=1).detach().cpu())  # [*, C]

                next_event_type_list.append(y_pred.detach().cpu())

            # next time prediction
            if hasattr(model, 'pred_next_time'):
                # non_pad_mask = Utils.get_non_pad_mask(event_type).squeeze(-1)
                sse, sse_norm, sae = Utils.time_loss(
                    model.y_next_time, event_time, non_pad_mask)  # sse, sse_norm, sae
                # total_loss.append(sse*opt.w_time)
                total_time_sse += sse.item()  # cumulative time prediction squared-error
                total_time_sae += sae.item()  # cumulative time prediction squared-error
                # cumulative time prediction squared-error
                total_time_sse_norm += sse_norm.item()

            # label prediction
            if hasattr(model, 'pred_label') and (state_label is not None):

                # state_label_red = state_label.sum(1).bool().int()[:,None,None] # [B,1,1]
                # state_label_loss,(y_state_pred, y_state_true, y_state_score) = Utils.state_label_loss(state_label_red,model.y_label, non_pad_mask)

                state_label_red = align(
                    state_label[:, :, None], event_time, state_time)  # [B,L,1]
                # state_label_red = state_label.bool().int()[:,:,None] # [B,L,1]
                state_label_loss, (y_state_pred, y_state_true, y_state_score) = Utils.state_label_loss(
                    state_label_red, model.y_label, non_pad_mask, opt.label_loss_fun)

                # total_loss.append(state_label_loss*opt.w_sample_label)
                y_state_pred_list.append(torch.flatten(
                    y_state_pred).detach().cpu())  # [*]
                y_state_true_list.append(torch.flatten(
                    y_state_true).detach().cpu())  # [*]
                y_state_score_list.append(torch.flatten(
                    y_state_score).detach().cpu())  # [*] it is binary

    masks = torch.cat(masks_list).detach().cpu()  # [*]

    out['non_pad_mask_list'] = non_pad_mask_list

    out['event_type_list'] = event_type_list
    out['event_time_list'] = event_time_list
    out['next_event_type_list'] = next_event_type_list

    out['list_intens_at_samples'] = list_intens_at_samples
    out['list_taus'] = list_taus
    out['list_true_intens_at_evs'] = list_true_intens_at_evs
    out['list_log_sum'] = list_log_sum
    out['list_integral_'] = list_integral_

    out['state_mod_list'] = state_mod_list
    out['state_time_list'] = state_time_list
    out['state_value_list'] = state_value_list

    out['r_enc_list'] = r_enc_list

    out['list_TE_att'] = list_TE_att
    out['list_DAM_att'] = list_DAM_att

    # CIF decoder
    if hasattr(model, 'event_decoder'):

        dict_metrics.update({
            'CIF/LL-#events': total_event_ll.item() / total_num_pred,
            'CIF/NLL': -total_event_ll.item(),
            'CIF/#events': total_num_pred,
        })

    # next time prediction
    if hasattr(model, 'pred_next_time'):
        rmse = np.sqrt(total_time_sse / total_num_pred)
        msae = total_time_sae / total_num_pred
        rmse_norm = np.sqrt(total_time_sse_norm / total_num_pred)
        dict_metrics.update({
            'NextTime/RMSE': rmse,
            'NextTime/rmse_norm': rmse_norm,
            'NextTime/msae': msae,
        })

    # next type prediction
    if hasattr(model, 'pred_next_type'):

        if y_pred_list[-1].dim() == 2:  # multilabel or marked
            y_pred = (np.concatenate(y_pred_list)[masks, :])
            y_true = (np.concatenate(y_true_list)[masks, :])
            y_score = (np.concatenate(y_score_list)[masks, :])

            bad_labels = y_true.sum(0) == 0

            y_true = y_true[:, ~bad_labels]
            y_pred = y_pred[:, ~bad_labels]
            y_score = y_score[:, ~bad_labels]
            n_classes = y_true.shape[1]

            cm = metrics.multilabel_confusion_matrix(y_true, y_pred)
            cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix=cm)

            dict_metrics.update({

                'NextType(ML)/auc-ovo-weighted': metrics.roc_auc_score(y_true, y_score, multi_class='ovo', average='weighted'),
                # # 'NextType(ML)/auc-ovo-micro': metrics.roc_auc_score(y_true, y_score, multi_class='ovo',average='micro'),
                # # 'NextType(ML)/auc-ovo-macro': metrics.roc_auc_score(y_true, y_score, multi_class='ovo',average='macro'),

                'NextType(ML)/auc-PR-weighted': metrics.average_precision_score(y_true, y_score, average='weighted'),
                'NextType(ML)/f1-weighted': metrics.f1_score(y_true, y_pred, average='weighted', zero_division=0),
                'NextType(ML)/precision-weighted': metrics.precision_score(y_true, y_pred, average='weighted', zero_division=0),
                'NextType(ML)/recall-weighted': metrics.recall_score(y_true, y_pred, average='weighted', zero_division=0),

                # 'NextType(ML)/f1-micro': metrics.f1_score(y_true, y_pred ,average='micro', zero_division=0),
                # 'NextType(ML)/f1-macro': metrics.f1_score(y_true, y_pred ,average='macro', zero_division=0),
                # 'NextType(ML)/f1-samples': metrics.f1_score(y_true, y_pred ,average='samples', zero_division=0),

                # 'NextType(ML)/acc': metrics.accuracy_score(y_true, y_pred),
                # 'NextType(ML)/hamming': metrics.hamming_loss(y_true, y_pred),

                # 'NextType(ML)/precision-micro': metrics.precision_score(y_true, y_pred ,average='micro', zero_division=0),
                # 'NextType(ML)/precision-macro': metrics.precision_score(y_true, y_pred ,average='macro', zero_division=0),
                # 'NextType(ML)/precision-samples': metrics.precision_score(y_true, y_pred ,average='samples', zero_division=0),

                # 'NextType(ML)/recall-micro': metrics.recall_score(y_true, y_pred ,average='micro', zero_division=0),
                # 'NextType(ML)/recall-macro': metrics.recall_score(y_true, y_pred ,average='macro', zero_division=0),
                # 'NextType(ML)/recall-samples': metrics.recall_score(y_true, y_pred ,average='samples', zero_division=0),

                # 'NextType(ML)/auc-ovo-macro': metrics.roc_auc_score(y_true, y_score, multi_class='ovo',average='macro'),
                # 'NextType(ML)/auc-ovo-micro': metrics.roc_auc_score(y_true, y_score, multi_class='ovo',average='micro'),
                # 'NextType(ML)/auc-PR-micro': metrics.average_precision_score(y_true, y_score ,average='micro'),
                # 'NextType(ML)/auc-PR-macro': metrics.average_precision_score(y_true, y_score ,average='macro'),

                # 'MultiLabel/AUROC-ovo-weighted': metrics.roc_auc_score(y_true, y_score, multi_class='ovo',average='weighted'),
                # 'MultiLabel/AUPRC-weighted': metrics.average_precision_score(y_true, y_score ,average='weighted'),
                # 'MultiLabel/F1-weighted': metrics.f1_score(y_true, y_pred ,average='weighted', zero_division=0),

                # 'ConfMat': cm_display,

            })

            if hasattr(model, 'event_decoder') and model.event_decoder.n_cifs == n_classes:

                y_event_score = (np.concatenate(y_event_score_list)[masks, :])

                # y_event_score = nn.functional.normalize(y_event_score,p=1,dim=1)
                y_event_pred = (y_event_score > 0.5).astype(int)

                y_event_pred = y_event_pred[:, ~bad_labels]
                y_event_score = y_event_score[:, ~bad_labels]

                dict_metrics.update({

                    'NextType(ML)/auc-ovo-weighted-CIF': metrics.roc_auc_score(y_true, y_event_score, multi_class='ovo', average='weighted'),
                    'NextType(ML)/f1-weighted-CIF': metrics.f1_score(y_true,  y_event_pred, average='weighted', labels=torch.arange(n_classes)),
                })

        else:   # multiclass
            y_pred = (np.concatenate(y_pred_list)[masks])
            y_true = (np.concatenate(y_true_list)[masks])
            y_score = (np.concatenate(y_score_list)[masks, :])

            cm = metrics.confusion_matrix(y_true, y_pred)
            cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix=cm)

            if hasattr(model, 'event_decoder') and model.event_decoder.n_cifs == n_classes:

                y_event_score = (np.concatenate(y_event_score_list)[masks, :])
                y_event_score = y_event_score/y_event_score.sum(1)[:, None]
                y_event_pred = np.argmax(y_event_score, 1)

                if n_classes == 2:

                    dict_metrics.update({

                        'NextType(MC)/auc-ovo-weighted-CIF': metrics.roc_auc_score(y_true, y_score[:, 0], multi_class='ovo', average='weighted'),
                        'NextType(MC)/f1-weighted-CIF': metrics.f1_score(y_true,  y_event_pred, average='weighted', zero_division=0),
                    })
                else:
                    dict_metrics.update({

                        'NextType(MC)/auc-ovo-weighted-CIF': metrics.roc_auc_score(y_true, y_event_score, multi_class='ovo', average='weighted', labels=torch.arange(n_classes), ),
                        'NextType(MC)/f1-weighted-CIF': metrics.f1_score(y_true,  y_event_pred, average='weighted', labels=torch.arange(n_classes), zero_division=0),
                    })

            if n_classes == 2:
                dict_metrics.update({
                    # 'NextType(MC)/f1-micro': metrics.f1_score(y_true, y_pred, labels= torch.arange(n_classes) ,average='micro', zero_division=0),
                    # 'NextType(MC)/f1-macro': metrics.f1_score(y_true, y_pred,average='weighted'),
                    'NextType(MC)/f1-weighted': metrics.f1_score(y_true, y_pred, average='weighted'),
                    'NextType(MC)/precision-weighted': metrics.precision_score(y_true, y_pred),
                    'NextType(MC)/recall-weighted': metrics.recall_score(y_true, y_pred),

                    # 'NextType(MC)/auc-ovo-macro': metrics.roc_auc_score(y_true, y_score[:,0], multi_class='ovo',average='macro'),
                    'NextType(MC)/auc-weighted': metrics.roc_auc_score(y_true, y_score[:, 0], multi_class='ovo', average='weighted'),
                    # 'auc-ovo-weighted-stupid': metrics.roc_auc_score(y_true, y_pred_stupid, multi_class='ovo',average='weighted',labels= torch.arange(n_classes)),



                    'NextType(MC)/acc': metrics.accuracy_score(y_true, y_pred, normalize=True),

                    'ConfMat': cm_display,


                })
            else:
                dict_metrics.update({
                    # 'NextType(MC)/f1-micro': metrics.f1_score(y_true, y_pred, labels= torch.arange(n_classes) ,average='micro', zero_division=0),
                    'NextType(MC)/f1-macro': metrics.f1_score(y_true, y_pred, labels=torch.arange(n_classes), average='macro', zero_division=0),
                    'NextType(MC)/f1-weighted': metrics.f1_score(y_true, y_pred, labels=torch.arange(n_classes), average='weighted', zero_division=0),

                    # 'NextType(MC)/auc-ovo-macro': metrics.roc_auc_score(y_true, y_score, multi_class='ovo',average='macro',labels= torch.arange(n_classes)),
                    'NextType(MC)/auc-weighted': metrics.roc_auc_score(y_true, y_score, multi_class='ovo', average='weighted', labels=torch.arange(n_classes)),
                    # 'auc-ovo-weighted-stupid': metrics.roc_auc_score(y_true, y_pred_stupid, multi_class='ovo',average='weighted',labels= torch.arange(n_classes)),



                    'NextType(MC)/acc': metrics.accuracy_score(y_true, y_pred, normalize=True),

                    'ConfMat': cm_display,


                })

        out['y_pred'] = y_pred
        out['y_true'] = y_true
        out['y_score'] = y_score

        out['y_true_list'] = y_true_list
        out['masks'] = masks

    # label prediction
    if hasattr(model, 'pred_label'):

        # state_label_red = state_label.sum(1).bool().int()[:,None,None] # [B,1,1]
        # state_label_loss,(y_state_pred, y_state_true, y_state_score) = Utils.state_label_loss(state_label_red,model.y_label, non_pad_mask)
        # # total_loss.append(state_label_loss*opt.w_sample_label)
        # y_state_pred_list.append(y_state_pred) # [*]
        # y_state_true_list.append(y_state_true) # [*]
        # y_state_score_list.append(y_state_score) # [*] it is binary

        # y_state_pred = (np.concatenate(y_state_pred_list)) # [*]
        # y_state_true = (np.concatenate(y_state_true_list))
        # y_state_score = (np.concatenate(y_state_score_list))

        # y_state_pred = (np.concatenate(y_state_pred_list) [masks]) # [*]
        # y_state_true = (np.concatenate(y_state_true_list) [masks])
        # y_state_score = (np.concatenate(y_state_score_list) [masks])

        y_state_pred = (np.concatenate(y_state_pred_list))  # [*]
        y_state_true = (np.concatenate(y_state_true_list))
        y_state_score = (np.concatenate(y_state_score_list))

        out['y_state_pred'] = y_state_pred
        out['y_state_true'] = y_state_true
        out['y_state_score'] = y_state_score

        dict_metrics.update({
            'pred_label/AUROC': metrics.roc_auc_score(y_state_true, y_state_score),
            'pred_label/AUPRC': metrics.average_precision_score(y_state_true, y_state_score),
            'pred_label/f1-binary': metrics.f1_score(y_state_true, y_state_pred, average='binary', zero_division=0),

            'pred_label/loss': total_label_state/total_num_event,
            'pred_label/recall-binary': metrics.recall_score(y_state_true, y_state_pred, average='binary', zero_division=0),
            'pred_label/precision-binary': metrics.precision_score(y_state_true, y_state_pred, average='binary', zero_division=0),

            'pred_label/ACC': metrics.accuracy_score(y_state_true, y_state_pred),


            # 'pred_label/sum_r_enc':torch.concat(r_enc_list, dim=1).sum().item(),
            # 'pred_label/r_enc_zero': a,
        })

    return dict_metrics, out

In [86]:
print('hi')
api = wandb.Api()
runs = api.runs("hokarami/TEEDAM_supervised")
df_filt = dl_runs(runs, selected_tag='A5')
len(df_filt)

# obtaining runs
df_config = pd.DataFrame([{k: v for k, v in x.items()}
                            for x in df_filt.config])
df_summary = pd.DataFrame([{k: v for k, v in x.items()}
                            for x in df_filt.summary])
df_path = df_filt.path.apply(lambda x: '/'.join(x))
df_con = pd.concat([df_config, df_summary, df_path], axis=1)
len(df_con)

df_con['transfer_learning'].unique()

if 'knn-ps-mean' in df_con:
    q = ((df_con['transfer_learning'] == 'DO') & (df_con['knn-ps-mean'].isnull())
            ) | ((df_con['INPUT'] == 'DAM') & (df_con['knn-ps-mean'].isnull()))
else:
    q = df_con['transfer_learning'].astype(bool)+True
q = df_con['transfer_learning'].astype(bool)+True

df_con = df_con[q].iloc[:]
len(df_con)
run_paths = df_con.path.tolist()

hi


2

2

array(['', 'DO'], dtype=object)

2

In [87]:
len(run_paths)

2

In [97]:

# run_paths = ["hokarami/TEEDAM_supervised/3drow7po"]
n_knn = 10
temp = list()
bad_runs = list()
bad_cal_sim = 0
run_path = run_paths[1]
api = wandb.Api()
run = api.run(run_path)

model1, opt1, run1 = read_from_wandb(
    run_path, consider_sample_labels=True)

dict_metrics1, out1 = valid_epoch_tsne(
    model1, opt1.validloader, opt1.pred_loss_func, opt1)
# X_tsne1, X_tsne_split1 = compute_tsne(out1['r_enc_list'], model1)
res_labels = opt1.dict_map_events.keys()
df1 = build_df(out1, opt1, X_tsne=None)

df1, bad_pids = remove_ids_with_no_pattern(df1, out1)

pid_all = list(df1.id)

pid_positive = list(df1[df1.color_true == 'Positive Samples'].id)

list_sim_score1 = []
list_t_max_rmse1 = []

list_sim_score2 = []
list_t_max_rmse2 = []
# X1 = np.concatenate(out1['r_enc_list'],axis=0)[:,:]

# only DAM embeddings
X2 = np.concatenate(out1['r_enc_list'], axis=0)[
    :, model1.d_out_te:model1.d_out_te+model1.d_out_dam]

# only TEE embeddings
X1 = np.concatenate(out1['r_enc_list'], axis=0)[:, :model1.d_out_te]

# TEE+DAM embeddings
X1 = np.concatenate(out1['r_enc_list'], axis=0)[
    :, :model1.d_out_te+model1.d_out_dam]

X1 = np.delete(X1, bad_pids, axis=0)
X2= np.delete(X2, bad_pids, axis=0)


for pid in tqdm(pid_positive):
    id_origin = pid

    knn_pids = find_knn_pids(X1, id_origin, n_knn=n_knn)
    try:
        sim_score1, list_summary, t_max_mae1 = cal_similarity(
            df1, out1, pid, knn_pids)

        if (not np.isnan(sim_score1)):
            list_sim_score1.append(sim_score1)
            list_t_max_rmse1.append(t_max_mae1)
    except:
        bad_cal_sim += 1





    # knn_pids = find_knn_pids(X2, id_origin, n_knn=n_knn)
    # try:
    #     sim_score1, list_summary, t_max_mae1 = cal_similarity(
    #         df1, out1, pid, knn_pids)

    #     if (not np.isnan(sim_score1)):
    #         list_sim_score2.append(sim_score1)
    #         list_t_max_rmse2.append(t_max_mae1)
    # except:
    #     pass


# np.mean(list_sim_score1), np.std(list_sim_score1)

run.summary['knn-ps-version'] = 1
run.summary['bad_pids'] = len(bad_pids)
run.summary['knn-ps-mean'] = np.mean(list_sim_score1)
run.summary['knn-ps-std'] = np.std(list_sim_score1)
run.summary['t_max_mae-mean'] = np.mean(list_t_max_rmse1)
run.summary['t_max_mae-std'] = np.std(list_t_max_rmse1)
run.summary.update()

temptemp = dict()
temptemp['path'] = run_path
temptemp['knn-ps-mean'] = np.mean(list_sim_score1)
temptemp['knn-ps-std'] = np.std(list_sim_score1)
temp.append(temptemp)

############################## CUDA True
[Info] INPUT TEDAM --> none-mark-label
[Info] parameters: Namespace(data='/mlodata1/hokarami/tedam/p12-raindrop/split0/', data_label='multilabel', cuda=1, wandb=True, wandb_project='TEEDAM_supervised', wandb_tag='None', log_freq=1, prof=False, per=100, balanced_batch=True, transfer_learning='DO', tl_tag='RD74-nextmark3', freeze=['TE'], ES_pat=100, setting='raindrop', test_center='', split='0', log='log.txt', user_prefix='[RD74-TL3-TEDA__nextmark-concat]', pos_alpha=1.0, epoch=50, batch_size=128, lr=0.001, smooth=0.0, weight_decay=0.1, diag_offset=0, event_enc=1, time_enc='concat', te_d_mark=8, te_d_time=4, te_d_rnn=256, te_d_inner=32, te_d_k=8, te_d_v=8, te_n_head=4, te_n_layers=4, te_dropout=0.1, dam_output_activation='relu', dam_output_dims=16, dam_n_phi_layers=3, dam_phi_width=128, dam_phi_dropout=0.2, dam_n_psi_layers=2, dam_psi_width=64, dam_psi_latent_width=128, dam_dot_prod_dim=64, dam_n_heads=4, dam_attn_dropout=0.1, dam_latent_width=64,

  - (Testing)   :   0%|          | 0/9 [00:00<?, ?it/s]

TOO BAD
TOO BAD
TOO BAD
TOO BAD
TOO BAD
TOO BAD
TOO BAD
TOO BAD


TOO BAD


0 patients were removed due to no existing pattern


100%|██████████| 188/188 [00:01<00:00, 115.40it/s]


In [96]:
#save dam
x2 = df1[df1.index.isin(pid_positive)].color_t_max.values
y2 = list_sim_score1#[:-2]
len(x2), len(y2)
np.mean(y2), np.std(y2)



(188, 188)

(0.8046125, 0.08883779)

In [98]:
# save tee-dam
x1 = df1[df1.index.isin(pid_positive)].color_t_max.values.copy()
y1 = list_sim_score1.copy()#[:-2]
len(x1), len(y1)
np.mean(y1), np.std(y1)


(188, 188)

(0.92654127, 0.032589737)

In [109]:
# x = df1[df1.index.isin(pid_positive)].color_t_max.values
# y = list_sim_score1#[:-2]
# z = list_t_max_rmse1#[:-2]


# y2 = list_sim_score2
# z2 = list_t_max_rmse2

# x.shape, len(y)
# plotly scatter plot. x-y and x-y2
fig = go.Figure()
_ = fig.add_trace(go.Scatter(x=x1, y=y1, mode='markers',
                         marker=dict(size=5, color='blue'),
                         name='TEE+DAM'))
_ =  fig.add_trace(go.Scatter(x=x2, y=y2, mode='markers',
                            marker=dict(size=5, color='red'),
                            name='DAM'))
_ =  fig.update_layout(title='Similarity score vs t_max',
                    xaxis_title='t_max(hours)',
                    yaxis_title='Similarity score')

# plot the means as horizontal dashed lines
_ = fig.add_shape(type="line", x0=20, y0=np.mean(y1), x1=np.max(x1), y1=np.mean(y1),
                line=dict(color="blue", width=1, dash="dash"))
_ = fig.add_shape(type="line", x0=20, y0=np.mean(y2), x1=np.max(x1), y1=np.mean(y2),
                line=dict(color="red", width=1, dash="dash"))
                  
fig.show()




In [80]:
x = df1[df1.index.isin(pid_positive)].color_t_max.values
y = list_sim_score1#[:-2]
z = list_t_max_rmse1#[:-2]


# y2 = list_sim_score2
# z2 = list_t_max_rmse2

x.shape, len(y)
# plotly scatter plot. x-y and x-y2
fig = go.Figure()
_ = fig.add_trace(go.Scatter(x=x, y=y, mode='markers',
                         marker=dict(size=5, color='blue'),
                         name='TEE+DAM'))
# _ =  fig.add_trace(go.Scatter(x=x, y=y2, mode='markers',
#                             marker=dict(size=5, color='red'),
#                             name='DAM'))
_ =  fig.update_layout(title='Similarity score vs t_max',
                    xaxis_title='t_max',
                    yaxis_title='Similarity score')
fig.show()

# scatter plot for z y
fig = px.scatter(x=z, y=y, labels={'x': 't_max', 'y': 'similarity score'})
fig.show()



((188,), 188)

In [58]:
df1.shape
len(list_sim_score1)
len(bad_pids)
X1.shape
len(pid_positive)

(1152, 9)

148

0

(1152, 140)

148

In [36]:
temptemp
out1.keys()
len(out1['event_type_list'])
out1['event_type_list'][0].shape
np.concatenate(out1['r_enc_list'], axis=0)[
    :, model1.d_out_te:model1.d_out_te+model1.d_out_dam].shape

{'path': 'hokarami/TEEDAM_supervised/2n678uyo',
 'knn-ps-mean': 0.92488325,
 'knn-ps-std': 0.03972259}

dict_keys(['non_pad_mask_list', 'event_type_list', 'event_time_list', 'next_event_type_list', 'list_intens_at_samples', 'list_taus', 'list_true_intens_at_evs', 'list_log_sum', 'list_integral_', 'state_mod_list', 'state_time_list', 'state_value_list', 'r_enc_list', 'list_TE_att', 'list_DAM_att', 'y_pred', 'y_true', 'y_score', 'y_true_list', 'masks', 'y_state_pred', 'y_state_true', 'y_state_score'])

9

torch.Size([128, 82, 24])

(1152, 128)

In [5]:

# run_paths = ["hokarami/TEEDAM_supervised/3drow7po"]
n_knn = 10
temp = list()
bad_runs = list()
bad_cal_sim = 0
for run_path in tqdm(run_paths[:1], leave=False):
    api = wandb.Api()
    run = api.run(run_path)
    try:
        model1, opt1, run1 = read_from_wandb(
            run_path, consider_sample_labels=True)

        dict_metrics1, out1 = Main.valid_epoch_tsne(
            model1, opt1.validloader, opt1.pred_loss_func, opt1)
        # X_tsne1, X_tsne_split1 = compute_tsne(out1['r_enc_list'], model1)
        res_labels = opt1.dict_map_events.keys()
        df1 = build_df(out1, opt1, X_tsne=None)

        df1, bad_pids = remove_ids_with_no_pattern(df1, out1)

        pid_all = list(df1.id)

        pid_positive = list(df1[df1.color_true == 'Positive Samples'].id)

        list_sim_score1 = []
        list_t_max_rmse1 = []

        # X1 = np.concatenate(out1['r_enc_list'],axis=0)[:,:]

        # only DAM embeddings
        X1 = np.concatenate(out1['r_enc_list'], axis=0)[
            :, model1.d_out_te:model1.d_out_te+model1.d_out_dam]

        # only TEE embeddings
        X1 = np.concatenate(out1['r_enc_list'], axis=0)[:, :model1.d_out_te]

        # TEE+DAM embeddings
        X1 = np.concatenate(out1['r_enc_list'], axis=0)[
            :, :model1.d_out_te+model1.d_out_dam]
        X1 = np.delete(X1, bad_pids, axis=0)

        for pid in tqdm(pid_positive):
            id_origin = pid

            knn_pids = find_knn_pids(X1, id_origin, n_knn=n_knn)
            try:
                sim_score1, list_summary, t_max_mae1 = cal_similarity(
                    df1, out1, pid, knn_pids)

                if (not np.isnan(sim_score1)):
                    list_sim_score1.append(sim_score1)
                    list_t_max_rmse1.append(t_max_mae1)
            except:
                bad_cal_sim += 1

        # np.mean(list_sim_score1), np.std(list_sim_score1)

        run.summary['knn-ps-version'] = 1
        run.summary['bad_pids'] = len(bad_pids)
        run.summary['knn-ps-mean'] = np.mean(list_sim_score1)
        run.summary['knn-ps-std'] = np.std(list_sim_score1)
        run.summary['t_max_mae-mean'] = np.mean(list_t_max_rmse1)
        run.summary['t_max_mae-std'] = np.std(list_t_max_rmse1)
        run.summary.update()

        temptemp = dict()
        temptemp['path'] = run_path
        temptemp['knn-ps-mean'] = np.mean(list_sim_score1)
        temptemp['knn-ps-std'] = np.std(list_sim_score1)
        temp.append(temptemp)
    except:
        bad_runs.append(run_path)
        run.summary['knn-ps-mean'] = -0.01
        continue


  0%|          | 0/80 [00:00<?, ?it/s]

############################## CUDA True
[Info] INPUT TEDAM --> none-mark-label
[Info] parameters: Namespace(data='/mlodata1/hokarami/tedam/p19-raindrop/split4/', data_label='multilabel', cuda=1, wandb=True, wandb_project='TEEDAM_supervised', wandb_tag='RD75', log_freq=1, prof=False, per=100, balanced_batch=True, transfer_learning='', tl_tag='None', freeze='', ES_pat=100, setting='raindrop', test_center='', split='4', log='log.txt', user_prefix='[RD74-TEDA__nextmark-concat]', pos_alpha=1.0, epoch=50, batch_size=128, lr=0.001, smooth=0.0, weight_decay=0.1, diag_offset=0, event_enc=1, time_enc='concat', te_d_mark=8, te_d_time=4, te_d_rnn=256, te_d_inner=32, te_d_k=8, te_d_v=8, te_n_head=4, te_n_layers=4, te_dropout=0.1, dam_output_activation='relu', dam_output_dims=16, dam_n_phi_layers=3, dam_phi_width=128, dam_phi_dropout=0.2, dam_n_psi_layers=2, dam_psi_width=64, dam_psi_latent_width=128, dam_dot_prod_dim=64, dam_n_heads=4, dam_attn_dropout=0.1, dam_latent_width=64, dam_n_rho_layers=2,

  1%|▏         | 1/80 [00:43<57:43, 43.84s/it]

############################## CUDA True
[Info] INPUT TEDAM --> none-mark-label
[Info] parameters: Namespace(data='/mlodata1/hokarami/tedam/p12-raindrop/split4/', data_label='multilabel', cuda=1, wandb=True, wandb_project='TEEDAM_supervised', wandb_tag='RD75', log_freq=1, prof=False, per=100, balanced_batch=True, transfer_learning='', tl_tag='None', freeze='', ES_pat=100, setting='raindrop', test_center='', split='4', log='log.txt', user_prefix='[RD74-TEDA__nextmark-concat]', pos_alpha=1.0, epoch=50, batch_size=128, lr=0.001, smooth=0.0, weight_decay=0.1, diag_offset=0, event_enc=1, time_enc='concat', te_d_mark=8, te_d_time=4, te_d_rnn=256, te_d_inner=32, te_d_k=8, te_d_v=8, te_n_head=4, te_n_layers=4, te_dropout=0.1, dam_output_activation='relu', dam_output_dims=16, dam_n_phi_layers=3, dam_phi_width=128, dam_phi_dropout=0.2, dam_n_psi_layers=2, dam_psi_width=64, dam_psi_latent_width=128, dam_dot_prod_dim=64, dam_n_heads=4, dam_attn_dropout=0.1, dam_latent_width=64, dam_n_rho_layers=2,

  2%|▎         | 2/80 [01:02<37:53, 29.15s/it]

...

############################## CUDA True
[Info] INPUT TEDAM --> ml-mark-label
[Info] parameters: Namespace(data='/mlodata1/hokarami/tedam/p19-raindrop/split4/', data_label='multilabel', cuda=1, wandb=True, wandb_project='TEEDAM_supervised', wandb_tag='RD75', log_freq=1, prof=False, per=100, balanced_batch=True, transfer_learning='', tl_tag='None', freeze='', ES_pat=100, setting='raindrop', test_center='', split='4', log='log.txt', user_prefix='[RD74-TEDA__pp_ml-concat]', pos_alpha=1.0, epoch=50, batch_size=128, lr=0.001, smooth=0.0, weight_decay=0.1, diag_offset=0, event_enc=1, time_enc='concat', te_d_mark=8, te_d_time=4, te_d_rnn=256, te_d_inner=32, te_d_k=8, te_d_v=8, te_n_head=4, te_n_layers=4, te_dropout=0.1, dam_output_activation='relu', dam_output_dims=16, dam_n_phi_layers=3, dam_phi_width=128, dam_phi_dropout=0.2, dam_n_psi_layers=2, dam_psi_width=64, dam_psi_latent_width=128, dam_dot_prod_dim=64, dam_n_heads=4, dam_attn_dropout=0.1, dam_latent_width=64, dam_n_rho_layers=2, dam_

  5%|▌         | 4/80 [05:16<2:12:23, 104.52s/it]

############################## CUDA True
[Info] INPUT TEDAM --> none-mark-label
[Info] parameters: Namespace(data='/mlodata1/hokarami/tedam/p19-raindrop/split4/', data_label='multilabel', cuda=1, wandb=True, wandb_project='TEEDAM_supervised', wandb_tag='RD75', log_freq=1, prof=False, per=100, balanced_batch=True, transfer_learning='', tl_tag='None', freeze='', ES_pat=100, setting='raindrop', test_center='', split='4', log='log.txt', user_prefix='[RD74-TEDA__none-concat]', pos_alpha=1.0, epoch=50, batch_size=128, lr=0.001, smooth=0.0, weight_decay=0.1, diag_offset=0, event_enc=1, time_enc='concat', te_d_mark=8, te_d_time=4, te_d_rnn=256, te_d_inner=32, te_d_k=8, te_d_v=8, te_n_head=4, te_n_layers=4, te_dropout=0.1, dam_output_activation='relu', dam_output_dims=16, dam_n_phi_layers=3, dam_phi_width=128, dam_phi_dropout=0.2, dam_n_psi_layers=2, dam_psi_width=64, dam_psi_latent_width=128, dam_dot_prod_dim=64, dam_n_heads=4, dam_attn_dropout=0.1, dam_latent_width=64, dam_n_rho_layers=2, dam

  6%|▋         | 5/80 [06:04<1:45:00, 84.00s/it] 

############################## CUDA True
[Info] INPUT DAM --> none-mark-label
[Info] parameters: Namespace(data='/mlodata1/hokarami/tedam/p19-raindrop/split4/', data_label='multilabel', cuda=1, wandb=True, wandb_project='TEEDAM_supervised', wandb_tag='RD75', log_freq=1, prof=False, per=100, balanced_batch=True, transfer_learning='', tl_tag='None', freeze='', ES_pat=100, setting='raindrop', test_center='', split='4', log='log.txt', user_prefix='[RD74-DA__base-concat]', pos_alpha=1.0, epoch=50, batch_size=128, lr=0.001, smooth=0.0, weight_decay=0.1, diag_offset=0, event_enc=0, time_enc='concat', te_d_mark=8, te_d_time=4, te_d_rnn=256, te_d_inner=32, te_d_k=8, te_d_v=8, te_n_head=4, te_n_layers=4, te_dropout=0.1, dam_output_activation='relu', dam_output_dims=16, dam_n_phi_layers=3, dam_phi_width=128, dam_phi_dropout=0.2, dam_n_psi_layers=2, dam_psi_width=64, dam_psi_latent_width=128, dam_dot_prod_dim=64, dam_n_heads=4, dam_attn_dropout=0.1, dam_latent_width=64, dam_n_rho_layers=2, dam_rho